### Datasets
http://www.manythings.org/anki/  (Download and unzip mar-eng.zip file)

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
lines= pd.read_table('mar.txt', names=['eng', 'mar'])

In [3]:
lines.shape

(33725, 2)

In [4]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [5]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [6]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [7]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [8]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [9]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [10]:
lines.sample(10)

,eng,mar
33248,more than one hundred nations have approved th...,START_ शंभरपेक्षा जास्त देशांनी कराराला मान्यत...
10102,its only temporary,START_ हे बस तात्पुरतं आहे _END
149,its hot,START_ गरम आहे _END
9371,can you play guitar,START_ तुम्ही गिटार वाजवू शकता का _END
16319,did you get my messages,START_ तुम्हाला माझे निरोप मिळाले का _END
7899,he opened the door,START_ त्याने दार उघडला _END
16726,i turned the lights out,START_ मी लाईट बंद केले _END
11682,i live in this hotel,START_ मी या हॉटेलमध्ये राहते _END
24674,im sending her to california,START_ मी तिला कॅलिफोर्नियाला पाठवतेय _END
8567,that was my finger,START_ ते माझं बोट होतं _END


In [11]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [12]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

34

In [13]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

37

In [14]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5388, 12709)

In [15]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

12710

In [16]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [17]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [18]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
12275,they are vegetarians,START_ ते शाकाहारी आहेत _END
7303,well do the rest,START_ बाकीचं आम्ही करू _END
16944,my brother plays guitar,START_ माझा भाऊ गिटार वाजवतो _END
19167,are these all toms books,START_ ही सर्व टॉमची पुस्तकं आहेत का _END
14094,this is not important,START_ हे महत्त्वाचं नाही आहे _END
8247,im good at tennis,START_ मी टेनिस चांगलं खेळते _END
24070,tom wanted to talk with mary,START_ टॉमला मेरीबरोबर बोलायचं होतं _END
5570,tom is my friend,START_ टॉम माझा मित्र आहे _END
2450,everybody lies,START_ सगळेच खोटं बोलतात _END
11503,how did he come here,START_ तो इथे कसा आला _END


In [19]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((30352,), (3373,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [20]:
## for training only
# X_train.to_pickle('Weights_Mar/X_train.pkl')
# X_test.to_pickle('Weights_Mar/X_test.pkl')

In [21]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [22]:
latent_dim = 50

In [23]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens + 1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [24]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [26]:
#from IPython.display import Image
#Image(retina=True, filename='train_model.png')

In [27]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [28]:
## only for training
"""
import sys
oldStdout = sys.stdout
file = open('logFile', 'w')
sys.stdout = file
"""

"\nimport sys\noldStdout = sys.stdout\nfile = open('logFile', 'w')\nsys.stdout = file\n"

In [29]:
## only for training
"""model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)"""

'model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),\n                    steps_per_epoch = train_samples//batch_size,\n                    epochs=epochs,\n                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),\n                    validation_steps = val_samples//batch_size)'

### Always remember to save the weights

In [30]:
## only for training
# model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [31]:
model.load_weights('nmt_weights.h5')

### Inference Setup

In [32]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [33]:
#Image(retina=True, filename='encoder_model.png')

In [34]:
#Image(retina=True, filename='decoder_model.png')

### Decode sample sequeces

In [35]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [36]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [37]:
## only for training
#sys.stdout = oldStdout

In [38]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: you look like my sister
Actual Marathi Translation:  तू माझ्या बहिणीसारखा दिसतोस 
Predicted Marathi Translation:  माझी आई खरच आवडतात 


In [39]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [40]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom is a few months older than mary
Actual Marathi Translation:  टॉम मेरीपेक्षा काही महिन्यांनी मोठा आहे 
Predicted Marathi Translation:  टॉम काही मेरीपेक्षा एक मोठी आहे 


In [41]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i cant see well
Actual Marathi Translation:  मला ठीकपणे बघता येत नाहिये 
Predicted Marathi Translation:  मला त्यांना बोलू शकत नाही 
